Init

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

spark = SparkSession.builder.appName("MostPopularSuperhero").getOrCreate()

schema = StructType([ \
                     StructField("id", IntegerType(), True), \
                     StructField("name", StringType(), True)])

names = spark.read.schema(schema).option("sep", " ").csv("./work/data/Marvel+names")

lines = spark.read.text("./work/data/Marvel+graph")

Aggregate connections

In [3]:
connections = (lines.withColumn(
        "id",
        func.split(
            func.trim(
                func.col("value")
                ),
                " "
            )[0]
    )
    .withColumn(
        "connections",
        func.size(
            func.split(
                func.trim(
                    func.col("value")
                ),
                " "
            )
        ) - 1
    )
    .groupBy("id")
    .agg(func.sum("connections").alias("connections")))

Most popular superhero

In [4]:
mostPopular = connections.sort(func.col("connections").desc()).first()

mostPopularName = names.filter(func.col("id") == mostPopular[0]).select("name").first()

print(mostPopularName[0] + " is the most popular superhero with " + str(mostPopular[1]) + " co-appearances.")

CAPTAIN AMERICA is the most popular superhero with 1933 co-appearances.


Obscure superheros

In [15]:
minConnectionsCount = connections.agg(func.min("connections")).first()[0]
leastConnections = connections.filter(func.col("connections") == minConnectionsCount)
connectionsWithNames = leastConnections.join(names, "id")

connectionsWithNames.select("name").show(connectionsWithNames.count())

+--------------------+
|                name|
+--------------------+
|        BERSERKER II|
|              BLARE/|
|MARVEL BOY II/MARTIN|
|MARVEL BOY/MARTIN BU|
|      GIURESCU, RADU|
|       CLUMSY FOULUP|
|              FENRIS|
|              RANDAK|
|           SHARKSKIN|
|     CALLAHAN, DANNY|
|         DEATHCHARGE|
|                RUNE|
|         SEA LEOPARD|
|         RED WOLF II|
|              ZANTOR|
|JOHNSON, LYNDON BAIN|
|          LUNATIK II|
|                KULL|
|GERVASE, LADY ALYSSA|
+--------------------+

